In [ ]:
import pandas as pd
import re

log_file_path = "D:/text-normalization/data_label.log"

# List to store extracted data
extracted_data = []

# Open and read the log file
with open(log_file_path, 'r', encoding='utf-8') as log_file:
    for line in log_file:
        # Assuming each result is printed on a single line in a format like: labeled_data: {...}
        match = re.search(r'labeled_data:\s*(\{.*\})', line)  # Adjust the regex pattern based on the log format
        if match:
            # Extract and process the labeled data
            labeled_data_str = match.group(1)
            # You can convert it to a dict or list depending on your data format (example assumes dict-like structure)
            labeled_data = eval(labeled_data_str)
            extracted_data.append(labeled_data)

# Convert the extracted data to a DataFrame and save to CSV
df = pd.DataFrame(extracted_data)
df.to_csv('labeled_data.csv', index=False)


In [ ]:
import re

# Path to your log file
log_file_path = "D:/text-normalization/data_label.log"

# List to store the extracted paragraphs
extracted_paragraphs = []

# Regex pattern to capture text between 'Finish processing paragraph' and 'INFO: ================================'
pattern = re.compile(r'Finish processing paragraph \d+\n(.*?)\n\d{2}:\d{2}:\d{2} INFO: ================================', re.DOTALL)

# Open and read the log file
with open(log_file_path, 'r', encoding='utf-8') as log_file:
    log_content = log_file.read()
    
    # Find all matches for the pattern
    matches = pattern.findall(log_content)
    
    # Store the extracted paragraphs
    extracted_paragraphs.extend(matches)

# Printing the extracted paragraphs (optional)
for i in range(5):
  print(extracted_paragraphs[i])

# You can then save the extracted paragraphs to a CSV file if needed


Trong thời gian gần đây, vấn đề tài chính cũng như vấn đề nguồn cầu thủ luôn là vấn đề đau đầu câu lạc bộ thành phố Hồ Chí Minh, nhất là khi thời hạn đăng ký tham dự mùa giải mới, cũng như thời hạn chứng minh năng lực tài chính mà liên đoàn bóng đá Việt Nam quy định đối với các đội bóng chuyên nghiệp gần đến (hạn chót là ngày ba mươi tháng mười).
Trong bối cảnh đó, câu lạc bộ thành phố Hồ Chí Minh vừa nhận được sự hỗ trợ cực kỳ kịp lúc từ phía người anh em cùng thành phố là xuân thành Sài Gòn. Theo đó, xuân thành Sài Gòn nhiều khả năng sẽ chi viện cho câu lạc bộ khoảng bảy đến tám cầu thủ, trong đó có cả ngoại binh em-ê-khà.
Những cầu thủ trẻ của xuân thành Sài Gòn như Tuấn Linh, Thế Sơn, Công Thành, Thứa Trí, Trọng Phi… tiến bộ rất nhanh trong thời gian vừa qua, tạo được một số dấu ấn tại vi-lích. Nên nếu những cầu thủ này khoác áo câu lạc bộ thành phố Hồ Chí Minh, thì đấy sẽ là sự bổ sung rất quý giá cho đội bóng thành phố.

Bầu Thụy (trái) bất ngờ quyết định hỗ trợ cho câu lạc bộ th

In [5]:
len(extracted_paragraphs) 
extracted_paragraphs[1500]

'Sau chức vô địch bun-de-liga và champions lếch, ba-yer munich bước vào trận chung kết đê-ê-ép cúp (cúp quốc gia đức) với cơ hội trở thành đội bóng đức đầu tiên làm nên cú ăn ba lịch sử. So với trận chung kết champions lếch, đội hình ra quân của ba-yer munich có hai sự thiếu vắng đáng kể khi trung vệ đan-te và tiền vệ lu-iz gu-ta-xtô không thể cùng đồng đội tham dự trận đấu tại béc-lin vì bận tập trung cùng đội tuyển bra-xin chuẩn bị cho côn-fed cúp.  \nBa-yer munich trải qua chín mươi phút nhọc nhằn trước s-tút-gác. Ngoài ra huấn luyện viên jupp hayn-ke-s cũng có sự thay đổi bất ngờ ở vị trí trung phong khi sử dụng ma-ri-ô go-mez thi đấu ngay từ đầu thay vì ma-ri-ô man-đu-kích. Tương tự như trận chung kết champions lếch, ba-yer munich là đội nhập cuộc chậm hơn và để cho đối phương lấn lướt về mặt thế trận lẫn số cơ hội ăn bàn. Khung thành của thủ môn no-uy-er liên tục bị chao đảo bởi những pha dàn xếp tấn công sắc sảo của s-tút-gác mà đáng kể nhất là hai pha dứt điểm của ma-xim

In [9]:
from typing import List
# from core.config.config import LABELED_DATA_DIR, LABELED_DATA_FILE
LABELED_DATA_DIR = "data_storage/processed"
LABELED_DATA_FILE = "processed_data.csv"
import os


def save_datas_to_file(labeled_datas: List) -> None:
      data_dict = {"labeled_datas": labeled_datas}
      
      df = pd.DataFrame(data_dict)
      if not os.path.exists(LABELED_DATA_DIR):
        os.makedirs(LABELED_DATA_DIR, exist_ok=True)
        
      df.to_csv(os.path.join(LABELED_DATA_DIR, LABELED_DATA_FILE), mode="a", index=False)

In [10]:
save_datas_to_file(extracted_paragraphs)